In [6]:
import numpy as np
import pandas as pd
from gensim.models import KeyedVectors
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import string

import tensorflow as tf
import os,datetime

import pickle
from nltk.stem import WordNetLemmatizer
import string
import nltk
import re

2024-10-28 20:59:04.411917: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1730167144.519757  110776 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1730167144.548152  110776 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-28 20:59:04.796236: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [20]:
df = pd.read_csv("../r4.2/http.csv",nrows=100000)
df

,id,date,user,pc,url,content
0,{V1Y4-S2IR20QU-6154HFXJ},01/02/2010 06:55:16,LRR0148,PC-4275,http://msn.com/The_Human_Centipede_First_Seque...,remain representatives consensus concert altho...
1,{Q5R1-T3EF87UE-2395RWZS},01/02/2010 07:00:13,NGF0157,PC-6056,http://urbanspoon.com/Plunketts_Creek_Loyalsoc...,festival off northwards than congestion partne...
2,{X9O1-O0XW52VO-5806RPHG},01/02/2010 07:03:46,NGF0157,PC-6056,http://aa.com/Rhodocene/rhodocenium/fhaavatqrf...,long away reorganized baldwin seth business 18...
3,{G5S8-U5OG04TE-5299CCTU},01/02/2010 07:05:26,IRM0931,PC-7188,http://groupon.com/Leonhard_Euler/leonhard/tne...,among german schwein experimental becomes prev...
4,{L0R4-A9DH29VP-4553AUWM},01/02/2010 07:05:52,IRM0931,PC-7188,http://flickr.com/Inauguration_of_Barack_Obama...,kate criteria j 2008 highest 12 include books ...
...,...,...,...,...,...,...
99995,{F2B4-J5HG21CP-2472KNWD},01/05/2010 08:24:11,JIM0095,PC-9328,http://city-data.com/No_Way_Out_2004/hotty/sre...,enlarged under generic advantage vision do any...
99996,{P4C0-H4AC44QZ-5898ATQE},01/05/2010 08:24:12,BBS0039,PC-9436,http://stubhub.com/Hoover_Dam/ickes/zbgbeplpyr...,an unknown afternoon dietary state law nationa...
99997,{U9Y2-H0WW90NF-1184ROHW},01/05/2010 08:24:12,KAL0395,PC-0004,http://tigerdirect.com/European_Commission/bar...,begin top we band themselves harshly or fourth...
99998,{Y2Y4-T1KW57MX-2017DWRE},01/05/2010 08:24:12,LDB0090,PC-6824,http://microsoft.com/Meteorological_history_of...,until acquire flared get secondary minas sea u...


In [26]:
# Load the Word2Vec model
word2vec_model = KeyedVectors.load_word2vec_format('../models/GoogleNews-vectors-negative300.bin.gz', binary=True)

In [22]:
# Download nltk data files (if not already installed)
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Error loading stopwords: <urlopen error [Errno -2] Name or
[nltk_data]     service not known>
[nltk_data] Error loading wordnet: <urlopen error [Errno -2] Name or
[nltk_data]     service not known>


False

In [23]:
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

In [24]:
def preprocess(text):
    # Lowercase the text and remove non-alphabetic characters
    text = re.sub(r'[^a-zA-Z\s]', '', text.lower())
    # Remove stopwords and lemmatize
    cleaned_text = ' '.join(lemmatizer.lemmatize(word) for word in text.split() if word not in stop_words)
    return cleaned_text

In [27]:
# Define function to get sentence vector
def get_sentence_vector(text):
    words = preprocess(text).split()  # Preprocess and split the text into words
    word_vectors = [word2vec_model[word] for word in words if word in word2vec_model]
    
    # If there are no valid words, return a zero vector
    if not word_vectors:
        return np.zeros(word2vec_model.vector_size)
    
    # Calculate the mean of word vectors for each sentence
    return np.mean(word_vectors, axis=0)

# Apply the function to your DataFrame
df['content_vector'] = df['content'].apply(get_sentence_vector)
df

,id,date,user,pc,url,content,new_content,content_vector
0,{V1Y4-S2IR20QU-6154HFXJ},01/02/2010 06:55:16,LRR0148,PC-4275,http://msn.com/The_Human_Centipede_First_Seque...,remain representatives consensus concert altho...,remain representative consensus concert althou...,"[0.025419906, 0.030988539, 0.0007039663, 0.048..."
1,{Q5R1-T3EF87UE-2395RWZS},01/02/2010 07:00:13,NGF0157,PC-6056,http://urbanspoon.com/Plunketts_Creek_Loyalsoc...,festival off northwards than congestion partne...,festival northward congestion partnership marr...,"[0.018577939, 0.038862318, -0.05105591, 0.0962..."
2,{X9O1-O0XW52VO-5806RPHG},01/02/2010 07:03:46,NGF0157,PC-6056,http://aa.com/Rhodocene/rhodocenium/fhaavatqrf...,long away reorganized baldwin seth business 18...,long away reorganized baldwin seth business tr...,"[-0.029726226, 0.024776326, -0.036115974, 0.05..."
3,{G5S8-U5OG04TE-5299CCTU},01/02/2010 07:05:26,IRM0931,PC-7188,http://groupon.com/Leonhard_Euler/leonhard/tne...,among german schwein experimental becomes prev...,among german schwein experimental becomes prev...,"[0.0021921794, 0.045175172, 0.00957133, 0.1425..."
4,{L0R4-A9DH29VP-4553AUWM},01/02/2010 07:05:52,IRM0931,PC-7188,http://flickr.com/Inauguration_of_Barack_Obama...,kate criteria j 2008 highest 12 include books ...,kate criterion j highest include book ensuring...,"[-0.053494874, 0.025620727, 0.0054260255, 0.01..."
...,...,...,...,...,...,...,...,...
99995,{F2B4-J5HG21CP-2472KNWD},01/05/2010 08:24:11,JIM0095,PC-9328,http://city-data.com/No_Way_Out_2004/hotty/sre...,enlarged under generic advantage vision do any...,enlarged generic advantage vision jamaica poss...,"[0.002129632, 0.02529165, -0.014264081, 0.0252..."
99996,{P4C0-H4AC44QZ-5898ATQE},01/05/2010 08:24:12,BBS0039,PC-9436,http://stubhub.com/Hoover_Dam/ickes/zbgbeplpyr...,an unknown afternoon dietary state law nationa...,unknown afternoon dietary state law national f...,"[-0.037862513, 0.056490157, 0.025104947, 0.064..."
99997,{U9Y2-H0WW90NF-1184ROHW},01/05/2010 08:24:12,KAL0395,PC-0004,http://tigerdirect.com/European_Commission/bar...,begin top we band themselves harshly or fourth...,begin top band harshly fourth sell first compo...,"[0.06907099, -0.014849576, -0.023220062, 0.057..."
99998,{Y2Y4-T1KW57MX-2017DWRE},01/05/2010 08:24:12,LDB0090,PC-6824,http://microsoft.com/Meteorological_history_of...,until acquire flared get secondary minas sea u...,acquire flared get secondary mina sea exercise...,"[0.04055214, 0.06912422, 0.0029149055, 0.08141..."


In [29]:
X_word2vec = np.array(df['content_vector'].tolist())  # Shape: (num_samples, 300)

# Assuming other features are already numeric, extract them as a NumPy array
X_other_features = df[['user', 'date']].values  # Adjust to your actual features

# Concatenate Word2Vec and other features along the last axis
X_combined = np.concatenate([X_word2vec, X_other_features], axis=1)
X_combined[0]

array([0.02541990578174591, 0.0309885386377573, 0.0007039663032628596,
       0.04840087890625, -0.09555961191654205, 0.016370102763175964,
       0.0021161260083317757, -0.12393930554389954, 0.1156497672200203,
       0.031455788761377335, -0.05752914026379585, -0.0925375446677208,
       -0.0032893000170588493, 0.052560340613126755, -0.09031883627176285,
       0.05859375, 0.008460585959255695, 0.09395867586135864,
       0.018667273223400116, -0.09227360785007477, 0.03021075204014778,
       0.11499765515327454, 0.0017114587826654315, 0.055021852254867554,
       -0.0023432550951838493, 0.02868487313389778, -0.06849001348018646,
       0.09488822519779205, 0.027063988149166107, -0.028739310801029205,
       -0.013997671194374561, -0.0015935124829411507,
       -0.030054040253162384, -0.018781404942274094, 0.03299115225672722,
       -0.005537909455597401, 0.027533041313290596, 0.0034196183551102877,
       0.014923095703125, 0.024471798911690712, 0.06668379157781601,
       -0.01417